In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from finetuning_dataset import QuickDrawFineTuningDataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

dataset = QuickDrawFineTuningDataset(labels=["apple", "cat"], tokenizer=tokenizer, max_length=1024)
model = AutoModelForCausalLM.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=2000,
    save_total_limit=200,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,               # only helps if on GPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

Loading QuickDraw files: 100%|██████████| 2/2 [00:05<00:00,  2.79s/it]


Loaded tokenized data from sketch_tokenized_dataset_test_gpt2.pkl


Step,Training Loss
